# 最优化算法

https://machine-learning-from-scratch.readthedocs.io/zh_CN/latest/%E6%9C%80%E4%BC%98%E5%8C%96%E7%AE%97%E6%B3%95.html#gradient-descent

![优化算法关系图](https://pic4.zhimg.com/v2-8626322a8a7a3ec0a5ad25bdee856d6d_r.jpg)

线上公式编辑器：https://www.codecogs.com/latex/eqneditor.php

- [为什么用梯度下降而不是牛顿法](#为什么用梯度下降而不是牛顿法)
- [1.梯度下降法](#1.梯度下降法)
    - [1.1批量梯度下降法(Batch_Gradient_Descent,BGD)](#1.1批量梯度下降法(Batch_Gradient_Descent,BGD))
    - [1.2随机梯度下降(Stochastic_Gradient_Descent,SGD)](#1.2随机梯度下降(Stochastic_Gradient_Descent,SGD))
    - [1.3小批量梯度下降法（Mini-batch_Gradient_Descent，MBGD）](#1.3小批量梯度下降法（Mini-batch_Gradient_Descent，MBGD）)
    - [1.4深度学习优化算法（动量项）](#1.4深度学习优化算法（动量项）)
        - [1.4.1Momentum](#1.4.1Momentum)
        - [1.4.2NAG（Nesterov_accelerated_gradient）](#1.4.2NAG（Nesterov_accelerated_gradient）)
    - [自适应学习率算法](#自适应学习率算法)
        - [1.4.3AdaGrad](#1.4.3AdaGrad)
        - [1.4.4Adadelta](#1.4.4Adadelta)
        - [1.4.5RMSProp](#1.4.5RMSProp)
        - [1.4.6Adam](#1.4.6Adam)
- [2.牛顿法和拟牛顿法](#2.牛顿法和拟牛顿法)
    - [2.1牛顿法](#2.1牛顿法)
        - [2.1.1用于方程求解](#2.1.1用于方程求解)
    - [2.2拟牛顿法](#2.2拟牛顿法)

## 为什么用梯度下降而不是牛顿法？


牛顿法：

- 二阶收敛，更快；
- 二次曲面拟合当前位置的局部曲面，选择的下降路径更符合真是的最优下降路径；

- 每一步都需要求解目标函数的Hessian矩阵的逆矩阵，计算比较复杂。

梯度下降法：

- 一阶收敛，较慢；
- 用平面拟合当前局部曲面；
- 易于并行。

- 接近全局最优解，但不是。


## 1.梯度下降法

最常用、一阶优化算法（只利用了一阶导数信息）、目标是凸函数、得到全局解。

一般情况下，不保证解是全局最优解，速度也未必是最快的。

需要假设函数是可微的，否则无法获得封闭解。

思想：用当前位置的负梯度方向作为搜索方向，移动与当前位置负梯度成比例的一段步长。因为该方向是当前位置的最快下降方向，所以也称为最速下降法。

 ![公式](https://machine-learning-from-scratch.readthedocs.io/zh_CN/latest/_images/%E6%A2%AF%E5%BA%A6%E4%B8%8B%E9%99%8D2.png)
 
 ![伪代码描述梯度下降](https://machine-learning-from-scratch.readthedocs.io/zh_CN/latest/_images/%E6%A2%AF%E5%BA%A6%E4%B8%8B%E9%99%8D3.png)

```python
class SGD: 
    def __init__(self, lr=0.01):
        self.lr = lr

    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads[key]

```

**缺点**：

- 靠近最优解的区域收敛速度明显变慢；

- 固定学习率的情况下，可能在某点附近震荡，如下例：

![震荡例子](https://machine-learning-from-scratch.readthedocs.io/zh_CN/latest/_images/%E6%A2%AF%E5%BA%A6%E4%B8%8B%E9%99%8D4.png)
![例图](https://machine-learning-from-scratch.readthedocs.io/zh_CN/latest/_images/%E6%A2%AF%E5%BA%A6%E4%B8%8B%E9%99%8D5.png)

从上图可知，固定learning rate时，如果learning rate太小，迭代越多，每次移动的距离越小，难以逼近最优值；lr太大，移动轨迹在某点附近开始震荡，之行移动。

解决办法：可变learning rate

### 1.1批量梯度下降法(Batch_Gradient_Descent,BGD)

批量梯度下降法是梯度下降法最原始的形式。

**最小化所有训练样本的损失函数，使得最终求解的是全局的最优解，即求解的参数是使得风险函数最小，但是对于大规模样本问题效率低下。**

**思路**：在更新每一参数都使用所有样本进行更新。

**优点**：

- 全局最优解；
- 易于并行实现；

**缺点**：

- 样本多的时候训练缓慢。

### 1.2随机梯度下降(Stochastic_Gradient_Descent,SGD)

最小化**每条样本**的损失函数，虽然**不是每次迭代得到的损失函数都向着全局最优方向**，但是大的**整体的方向是向全局最优解**的，最终的结果往往是在**全局最优解附近**，**适用于大规模训练样本**情况。

**思路**：在每次迭代时，只使用一个样本，当样本数量很大的时候，SGD迭代一次的速度远高于BGD。SGD以**损失一部分精确度**和**增加一定数量的迭代次数**为代价，换取**总体的优化效率的提升**。增加的迭代次数<<样本数量。若样本很多（例如几十万），可能用几万条或者几千条就迭代到最优解了。

优点：

- 训练速度快

缺点：

- 准确度下降，不是全局最优解


### 1.3小批量梯度下降法（Mini-batch_Gradient_Descent，MBGD）

**思路**：在更新每一参数时都使用一部分样本（batch）来进行更新，可以选择对每个 batch 的梯度进行累加，或者取平均值。取平均值可以减少梯度的方差。

克服了上面两种方法的缺点，又同时兼顾两种方法的优点，是如今深度学习领域最常见的实现方式。

- 准确度相较BGD有所下降，解接近全局最优解，但是训练速度比BGD快；
- 易于并行计算；


### 1.4深度学习优化算法（动量项）

![优化器效果图](../imgs/sgd.jpg)

![效果2](https://images2018.cnblogs.com/blog/1192699/201803/1192699-20180311105558593-251578131.gif)

![效果3](https://images2018.cnblogs.com/blog/1192699/201803/1192699-20180311110108768-2113908893.gif)

#### 优化器选择

https://www.cnblogs.com/guoyaohua/p/8542554.html

- 数据稀疏用自适用方法，即 Adagrad, Adadelta, RMSprop, Adam。

- RMSprop, Adadelta, Adam 在很多情况下的效果是相似的。

- Adam 就是在 RMSprop 的基础上加了 bias-correction 和 momentum。

- 随着梯度变的稀疏，Adam 比 RMSprop 效果会好。

- 整体来讲，Adam 是最好的选择。

- 很多论文里都会用 SGD，没有 momentum 等。SGD 虽然能达到极小值，但是比其它算法用的时间长，而且可能会被困在鞍点。

- 如果需要更快的收敛，或者是训练更深更复杂的神经网络，需要用一种自适应的算法。

----

#### 优化器粗解

https://cs231n.github.io/neural-networks-3/

- Momentum
    - 一阶优化算法
    - 动量替代梯度，本次动量=上次动量与当前梯度加权和（即累积梯度替代当前梯度）
    - 动量项累积了之前迭代时的梯度值，使得本次迭代时沿着之前的惯性方向向前走
    - 加快收敛速度，减少震荡——使得梯度方向不变的维度上速度变快，梯度方向有所改变的维度上的更新速度变慢，这样就可以加快收敛并减小震荡。
    - 缺点：依赖于鲁棒初始化，可能比其他优化器费时
```python
# Momentum update
# mu = 0.9
v = mu * v - learning_rate * dx # integrate velocity 存储梯度指数衰减平均值
x += v # integrate position
```

- NAG加速梯度
    - 一阶算法
    - 速度替代梯度，在计算梯度时，不是在当前位置，而是未来的位置上
    - 在批量梯度下降中，收敛速度比momentum更快；在SGD中无法调收敛速度
    - 预期的更新可以避免走的太快
```python
x_ahead = x + mu * v  # 在未来近似位置计算梯度，而不是x位置
# evaluate dx_ahead (the gradient at x_ahead instead of at x)
v = mu * v - learning_rate * dx_ahead
x += v
# =============================
v_prev = v # back this up
v = mu * v - learning_rate * dx # velocity update stays the same
x += -mu * v_prev + (1 + mu) * v # position update changes form
```

### 以下为自适应学习率的优化器方法

- Adagrad
    - 积累梯度的二次方调控学习率
    - 学习率基于参数进行调整，每次迭代中**每个参数使用的都是不同的学习率**,而不需要在学习过程中人为调整学习率。
    - 根据不常用（低频）的参数进行较大幅度的学习率更新,根据常用（高频）的参数进行较小幅度的学习率更新。因此，Adagrad 成了**稀疏数据**如图像识别和 NLP 的天然选择。
    - 缺点：分母会越来越大，导致lr趋于0，参数无法有效更新，提前结束训练。
    
```python
# Assume the gradient dx and parameter vector x
# learning_rate = 0.01
cache += dx**2  # 存储梯度的平方和
x += - learning_rate * dx / (np.sqrt(cache) + eps)
```

- Adadelta
    - 为了解决 Adagrad 学习率急剧下降问题的
    - 将adagrad中单纯叠加梯度的二次方改为梯度二次方的指数衰减平均值
    - 不需要设置学习率超参数

- RMSprop
    - 为了解决 Adagrad 学习率急剧下降问题的
    - 使用的是指数加权平均，旨在**消除梯度下降中的摆动**，与Momentum的效果一样，**某一维度的导数比较大，则指数加权平均就大，某一维度的导数比较小，则其指数加权平均就小**，这样就**保证了各维度导数都在一个量级，进而减少了摆动**。允许使用一个更大的学习率η

```python
# decay_rate = 0.9, learning_rate = 0.001
cache = decay_rate * cache + (1 - decay_rate) * dx**2  # 存储梯度指数衰减平均值
x += - learning_rate * dx / (np.sqrt(cache) + eps)  # 与adagrad相同
```

- Adam
    - 计算每个参数的自适应学习率的方法，相当于 RMSprop + Momentum
    - 除了像 Adadelta 和 RMSprop 一样**存储了过去梯度的平方 v 的指数衰减平均值**，也像 momentum 一样**保持了过去梯度 m 的指数衰减平均值**
    - 实践表明，Adam 比其他适应性学习方法效果要好
    - 缺点：可能陷入局部极小，泛化不好
    
```python
# beta1 = 0.9, beta2 = 0.999, eps = 10e-8
m = beta1*m + (1-beta1)*dx  # 存储过去梯度的指数衰减平均值，替代rmsprop中的梯度dx
v = beta2*v + (1-beta2)*(dx**2)  # 存储过去梯度的平方的指数衰减平均值

# 如果m和v初始化为0，则进行偏差矫正
# m = m/(1-beta1)
# v = v/(1-beta2)

x += - learning_rate * m / (np.sqrt(v) + eps)  # 与adagrad形式相同
```

#### 1.4.1Momentum

加快梯度下降法的收敛速度，减少震荡，引入了动量项。动量项累积了之前迭代时的梯度值，加上此项之后的参数更新公式为：

$$W_{t+1}=W_t + V_{t+1}$$

其中后一项$V_{t+1}$为动量项，它取代了之前的梯度项(梯度下降法公式每次加上当前负梯度值)。

动量项计算公式：

![动量项计算公式](https://pic1.zhimg.com/80/v2-f8704991e56c9890da923684ba69ebab_720w.jpg)

由公式可知，动量项是<font color=red>**上一时刻的动量项与本次梯度值的加权平均值**</font>(动量项累积了之前迭代时的梯度值，使得本次迭代时沿着之前的惯性方向向前走。)，其中 α 是学习率，μ 是动量项系数（初始设置0.5，稳定后改为0.9）。如果按照时间 t 进行展开，则第 t 次迭代时使用了从 1 到 t 次迭代时的所有梯度值，且旧的梯度值按照$\mu _t$的系数指数级衰减：

![动量项打开形式](https://picb.zhimg.com/80/v2-a2cf7ceb85faac443ba14aa3f2cd53d0_720w.jpg)



```python
class Momentum:
    def __init__(self, lr=0.01, momemtum=0.9):
        self.lr = lr  # α
        self.momemtum = momemtum  # μ
        self.v = None

    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():  # 每个可训练变量都拥有一个动量项
                self.v[key] = np.zeros_like(val)

        for key in params.keys():
            self.v[key] = self.momemtum * self.v[key] - self.lr * grads[key]  # 累积每次迭代的梯度值，根据当前梯度更新动量项
            params[key] += self.v[key]  # 用动量项更新可训练变量

```

#### 1.4.2NAG（Nesterov_accelerated_gradient）

momentum 保留了上一时刻的梯度 ![](https://www.zhihu.com/equation?tex=%5Ctriangledown_%7B%5Ctheta%7D+J+%5Cleft%28%5Ctheta+%5Cright%29) ，对其没有进行任何改变，NAG 是 momentum 的改进，在梯度更新时做一个矫正，具体做法就是在当前的梯度![](https://www.zhihu.com/equation?tex=%5Ctriangledown_%7B%5Ctheta%7D+J+%5Cleft%28%5Ctheta+%5Cright%29)上添加上一时刻的动量 $μ* m_t$ ，梯度改变为 $\triangledown _{\theta }J(\theta -\mu \cdot m_t)$ 。

公式如下：

$$m_{t+1} = \mu \cdot m_t + \alpha \cdot \bigtriangledown _{\theta }J(\theta -\mu \cdot m_t)$$

$$\theta _{t+1}=\theta _t + m_{t+1}$$

加上nesterov项后，梯度在大的跳跃后，对当前梯度进行校正。 下图是momentum和nesterrov的对比表述图：

![](https://pic1.zhimg.com/80/v2-51454083960f1826e2f5a5af2b6271cd_720w.jpg)

momentum 首先计算一个梯度(短的蓝色向量)，然后在加速更新梯度的方向进行一个大的跳跃(长的蓝色向量)，nesterov 项首先在之前加速的梯度方向进行一个大的跳跃(棕色向量)，计算梯度然后进行校正（红色向量），最后得到最终结果（绿色向量)

```python
class Nestrov:
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr  # α
        self.momentum = momentum  # μ
        self.v = None  # m

    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zeros_like(val)

        for key in params.keys():
            self.v[key] *= self.momentum  # μ*m_t
            self.v[key] -= self.lr * grads[key]  # μ*m_t-α*grads
            params[key] -= self.momentum * self.v[key]
            params[key] += (1 + self.momentum) * self.lr * grads[key]

```

### 自适应学习率算法

目前的自适应学习率优化算法主要有：AdaGrad算法，RMSProp算法，Adam算法以及AdaDelta算法。

#### 1.4.3AdaGrad 

SGD依赖人工设定lr，过大过小都不好。

思路：根据前几轮迭代时的历史梯度值动态调整学习率，且优化向量x的每一个分量$x_i$都有自己的学习率：

![adagrad公式](https://pic2.zhimg.com/80/v2-c65e5cd20f411c04547307d7944654ce_720w.jpg)

其中 α 是学习率，g_t是第 t 次迭代时参数的梯度向量， ε 是一个很小的正数，避免除数为0，下标 i 表示向量的分量。与 SGD 不同的是多了分母项。分母项累积到本次迭代为止梯度的历史值信息用于生成梯度下降的系数值。按照上式，历史导数值的绝对值越大，分量的学习率越小(避免震荡)，反之越大。

分步解释：
![](https://www.zhihu.com/equation?tex=%5C%5C++g+%5Cleftarrow++%5Ctriangledown_%7B%5Ctheta%7D+J+%5Cleft%28%5Ctheta+%5Cright%29+%5C%5C+++r+%5Cleftarrow+r+%2B+g%5E%7B2%7D+%5C%5C++%5Ctriangle+%5Ctheta+%5Cleftarrow++%5Cfrac%7B%5Cdelta+%7D%7B%5Csqrt%7Br+%2B+%5Cepsilon%7D%7D%5Ccdot+g+%5C%5C+%5Ctheta+%5Cleftarrow+%5Ctheta+-+%5Ctriangle+%5Ctheta+)

其中 r 即梯度加速变量。

**缺点**：

- 需要人工设置全局学习率α，如果过大使得正则化过于敏感，对梯度调节过大；
- 分母会越来越大，导致lr趋于0，参数无法有效更新，提前结束训练。

```python
class AdaGrad:
    def __init__(self, lr=0.01, epsilon=1e-7):
        self.lr = lr  # α
        self.r = None  # r
        self.epsilon = epsilon

    def update(self, params, grads):
        if self.r is None:
            self.r = {}
            for key, val in params.items():
                self.r[key] = np.zeros_like(val)

        for key in params.keys():
            self.r[key] += grads[key] * grads[key]  # 计算梯度加速变量
            params[key] -= self.lr * grads[key] / (np.sqrt(self.r[key]) + self.epsilon)  # 更新变量

```

#### 1.4.4Adadelta

问题：AdaGrad的学习率会不断地衰退。

思路：累加固定大小的项，不直接存储这些项，仅仅是近似计算对应的平均值，避免了长期累积梯度值所导致的学习率趋向于0的问题。

![adadelta公式](https://pic1.zhimg.com/80/v2-04c5ddde82db12bd6139c8d298c7165b_720w.jpg)
![](https://pic2.zhimg.com/80/v2-285b62e8956216546258e443bd87ba76_720w.jpg)
![](https://pic3.zhimg.com/80/v2-819cba3a32ba980692c34c62b3b6fcbd_720w.jpg)
![](https://picb.zhimg.com/80/v2-7d20d95f98741426ebcbfb39120b0882_720w.jpg)
![](https://pic1.zhimg.com/80/v2-20863539c3df7cc49f6443c591626a02_720w.jpg)
![](https://pic2.zhimg.com/80/v2-9ebe98ac6abb4cd2c6bac8b3ba24b9e8_720w.jpg)


特点：

- 经过近似牛顿法迭代后，不再依赖全局lr；
- 训练初中期，加速效果好，快；
- 训练后期，反复在局部最小附近抖动。

```python
# https://www.cnblogs.com/xiximayou/p/12713594.html
# 与公式不一致，需要进一步验证
class Adadelta():
    def __init__(self, rho=0.95, eps=1e-6):
        self.E_w_updt = None # Running average of squared parameter updates
        self.E_grad = None   # Running average of the squared gradient of w
        self.w_updt = None   # Parameter update
        self.eps = eps
        self.rho = rho

    def update(self, w, grad_wrt_w):  # w:x, grad_wrt_w:delta_x
        # If not initialized
        if self.w_updt is None:
            self.w_updt = np.zeros(np.shape(w))  # delta_x
            self.E_w_updt = np.zeros(np.shape(w))  # E_delta_x
            self.E_grad = np.zeros(np.shape(grad_wrt_w))  # E_g

        # Update average of gradients at w
        self.E_grad = self.rho * self.E_grad + (1 - self.rho) * np.power(grad_wrt_w, 2)
        
        RMS_delta_w = np.sqrt(self.E_w_updt + self.eps)  # RMS_delta_x
        RMS_grad = np.sqrt(self.E_grad + self.eps)  # RMS_g

        # Adaptive learning rate
        adaptive_lr = RMS_delta_w / RMS_grad

        # Calculate the update
        self.w_updt = adaptive_lr * grad_wrt_w

        # Update the running average of w updates
        self.E_w_updt = self.rho * self.E_w_updt + (1 - self.rho) * np.power(self.w_updt, 2)

        return w - self.w_updt
```

#### 1.4.5RMSProp

问题：AdaGrad有个问题，那就是学习率会不断地衰退。这样就会使得很多任务在达到最优解之前学习率就已经过量减小

思路：使用**指数衰减平均**慢慢**丢弃**先前得梯度历史，避免了长期累积梯度值所导致的学习率趋向于0的问题,在非凸设定下效果更好。

公式如下：

![rmsprop](https://www.zhihu.com/equation?tex=+g+%5Cleftarrow++%5Ctriangledown_%7B%5Ctheta%7D+J+%5Cleft%28%5Ctheta+%5Cright%29+%5C%5C+++E+%5Cleft%5B+g%5E%7B2%7D%5Cright%5D_%7Bt%7D+%5Cleftarrow+%5Crho+%5Ccdot+E+%5Cleft%5B+g%5E%7B2%7D%5Cright%5D_%7Bt-1%7D%2B+%5Cleft%281+-+%5Crho+%5Cright%29+%5Ccdot+g_%7Bt%7D%5E%7B2%7D+%5C%5C+%5Ctriangle+%5Ctheta+%5Cleftarrow++%5Cfrac%7B%5Cdelta+%7D%7B%5Csqrt%7BE+%5Cleft%5B+g%5E%7B2%7D%5Cright%5D_%7Bt%7D+%2B+%5Cepsilon%7D%7D%5Ccdot+g+%5C%5C++%5Ctheta+%5Cleftarrow+%5Ctheta+%2B+%5Ctriangle+%5Ctheta+)

特点：

- 依赖于全局lr；
- 较Adagrad增加了一个衰减系数，效果介于 AdaGrad 和 Adadelta之间；
- 适合处理非平稳目标——对rnn效果好；

```python
class RMSprop:
    def __init__(self, lr=0.01, decay_rate=0.99, epsilon=1e-7):
        self.lr = lr  # δ
        self.decay_rate = decay_rate  # ρ
        self.e = None
        self.eps = epsilon  # ε

    def update(self, params, grads):
        if self.e is None:
            self.e = {}
            for key, val in params.items():
                self.e[key] = np.zeros_like(val)  # rms向量初始化为0，按照衰减系数ρ累积历史的梯度平方值

        for key in params.keys():
            self.e[key] *= self.decay_rate  # ρ* E[g^2]
            self.e[key] += (1 - self.decay_rate) * grads[key] * grads[key]  # 更新rms向量
            params[key] -= self.lr * grads[key] / (np.sqrt(self.e[key]) + self.eps)  # 更新变量
```

#### 1.4.6Adam

思路：动量直接并入了梯度一阶矩（指数加权）的估计。其次，相比于缺少修正因子导致二阶矩估计可能在训练初期具有很高偏置的 RMSProp，Adam 包括偏置修正，修正从原点初始化的一阶矩（动量项）和（非中心的）二阶矩估计。

动量相当于给优化过程增加了惯性，自适应过程就像是给优化过程加入了阻力。速度越快，阻力也会越大。

![adam](https://www.zhihu.com/equation?tex=g+%5Cleftarrow+%5Ctriangledown_%7B%5Ctheta%7D+%5C%5C++J+%5Cleft%28%5Ctheta+%5Cright%29+%5C++m_%7Bt%7D+%5Cleftarrow+%5Cbeta_%7B1%7D+%5Ccdot+m_%7Bt-1%7D+%2B+%5Cleft%281+-+%5Cbeta_%7B1%7D+%5Cright%29+%5Ccdot+g_%7Bt%7D+%5C%5C++v_%7Bt%7D+%5Cleftarrow+%5Cbeta_%7B2%7D+%5Ccdot+v_%7Bt-1%7D+%2B+%5Cleft%28+1+-+%5Cbeta_%7B2%7D+%5Cright%29+%5Ccdot+g_%7Bt%7D%5E%7B2%7D++%5C%5C+%5Chat%7Bm%7D%7Bt%7D+%5Cleftarrow+%5Cfrac%7Bm%7Bt%7D%7D%7B1+-+%5Cbeta_%7B1%7D%5E%7Bt%7D%7D+%5C%5C++%5Chat%7Bv%7D%7Bt%7D+%5Cleftarrow+%5Cfrac%7Bv%7Bt%7D%7D%7B1+-+%5Cbeta_%7B2%7D%5E%7Bt%7D%7D+%5C%5C+%5Ctheta_%7Bt%2B1%7D+%3D+%5Ctheta_%7Bt%7D+-+%5Cfrac%7B%5Cdelta%7D%7B%5Cepsilon+%2B+%5Csqrt%7B%5Chat%7Bv_%7Bt%7D%7D%7D%7D+%5Ccdot+%5Chat%7Bm%7D_%7Bt%7D)

其中，m_t和n_t分别为梯度的一阶矩估计和二阶矩估计；对应带^的变量是偏差校正，如此可近似为对期望的无偏估计。

特点：

- 梯度经过偏置校正后，每次迭代lr都有一个固定范围，使得参数比较平稳；
- 结合Adagrad善于处理稀疏梯度和RMSProp善于处理非平稳目标的有点；
- 为不同参数计算不同的自适应学习率；
- 适用于大多数非凸优化——适用于大数据集和高维空间。

```python
class Adam:
    def __init__(self, lr=0.001, beta1=0.9, beta2=0.999):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.iter = 0
        self.m = None
        self.v = None

    def update(self, params, grads):
        if self.m is None:
            self.m, self.v = {}, {}
            for key, val in params.items():
                self.m[key] = np.zeros_like(val)  # 初始值都为0
                self.v[key] = np.zeros_like(val)

        self.iter += 1
        lr_t = self.lr * np.sqrt(1.0 - self.beta2**self.iter) / (1.0 - self.beta1**self.iter)

        for key in params.keys():
            
            # 由于m和v的初始指为0，所以第一轮的时候会非常偏向第二项，
            # 那么在后面计算更新值得时候根据β_1 与 β_2的初始值来看就会非常的大，需要将其修正回来。
            # 而且由于β_1 与 β_2很接近于1，所以如果不修正，对于最初的几轮迭代会有很严重的影响。
            self.m[key] += (1 - self.beta1) * (grads[key] - self.m[key])  # 修正一阶矩
            self.v[key] += (1 - self.beta2) * (grads[key]**2 - self.v[key])  # 修正二阶矩

            params[key] -= lr_t * self.m[key] / (np.sqrt(self.v[key]) + 1e-7)
```

## 2.牛顿法和拟牛顿法

二阶优化算法

核心：对函数进行泰勒展开

### 2.1牛顿法

#### 2.1.1用于方程求解

求解方程 f(x) = 0 的解：

1. 选择一个接近函数f(x)=0 处的 x0(在待求点x的一定区域内才必定收敛)，计算相应的f (x0) 和切线斜率f′(x0)

2. 计算过点(x0,f(x0)) 并且斜率为f′(x0) 的直线和 X 轴的交点的 x 坐标，也就是求如下方程的解：f(x0)+f′(x0)∗(x−x0)=0

3. 将新求得的点的 x 坐标命名为 x1，通常 x1 会比 x0 更接近方程f(x) = 0 的解。因此我们现在可以利用 x1 开始下一轮迭代。迭代公式可化简为如下所示：

![例子公式](https://machine-learning-from-scratch.readthedocs.io/zh_CN/latest/_images/%E7%89%9B%E9%A1%BF%E6%B3%951.png)

由于牛顿法是基于当前位置的切线来确定下一次的位置，所以牛顿法又被很形象地称为是”切线法”。

![牛顿法动图演示](https://machine-learning-from-scratch.readthedocs.io/zh_CN/latest/_images/%E7%89%9B%E9%A1%BF%E6%B3%952.gif)

或者下图：

![静图讲解牛顿法](https://machine-learning-from-scratch.readthedocs.io/zh_CN/latest/_images/%E7%89%9B%E9%A1%BF%E6%B3%953.png)

已经证明，如果f'是**连续**的，并且待求的零点x是**孤立**的，那么在零点x**周围**存在一个**区域**，只要**初始值 x0 位于这个邻近区域内**，那么牛顿法**必定收敛**。 并且，**如果f'(x)不为0**, 那么牛顿法将具有**平方收敛**的性能，这意味着每迭代一次，牛顿法结果的**有效数字将增加一倍**。

### 2.2拟牛顿法

**思路**：使用正定矩阵来近似Hessian矩阵的逆，简化运算的复杂度。

特点：只要求每一步迭代时知道目标函数的梯度，通过测量梯度的变化，构造一个目标函数的模型使之足以产生超线性收敛性。

解决无约束，约束，和大规模的优化问题